<a href="https://colab.research.google.com/github/zeineb-ouerghi/Capstone-Project-/blob/main/IRT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install py-irt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.3/197.3 KB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 43.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 730.7/730.7 KB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 KB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.6/197.6 KB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.6/196.6 KB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.6/196.6 KB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.5/196.5 KB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 KB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.6/195.6 KB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.6/195.6 KB 20.1 MB/s eta 0:00:00
     ━━━━━━━━

In [ ]:
pip install pyirt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pyirt import irt 

/usr/local/lib/python3.8/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /usr/local/lib/python3.8/dist-packages/pyirt/util/clib.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
## data acquisition 
## Spaced repetition DATA from Duolingo Research 
dataframe = pd.read_csv("/content/drive/MyDrive/Capstone Project/duolingodata.csv.gz")
#Data Cleaning and Manipulation 
#data.head()
#adding variables 
#adding number of mistakes made 
dataframe['mistakes'] = dataframe.history_seen - dataframe.history_correct
dataframe['session_mistakes'] = dataframe.session_seen - dataframe.session_correct
dataframe['rate_of_error_per_session'] = dataframe.session_mistakes/dataframe.session_seen
dataframe['general_rate_of_error'] = dataframe.mistakes/dataframe.history_seen 
# unit='s' to convert it into epoch time
dataframe['Datetime'] = pd.to_datetime(dataframe['timestamp'], 
                                  unit='s')
dataframe.head()

Mounted at /content/drive


,p_recall,timestamp,delta,user_id,learning_language,ui_language,lexeme_id,lexeme_string,history_seen,history_correct,session_seen,session_correct,mistakes,session_mistakes,rate_of_error_per_session,general_rate_of_error,Datetime
0,1.0,1362076081,27649635,u:FO,de,en,76390c1350a8dac31186187e2fe1e178,lernt/lernen<vblex><pri><p3><sg>,6,4,2,2,2,0,0.0,0.333333,2013-02-28 18:28:01
1,0.5,1362076081,27649635,u:FO,de,en,7dfd7086f3671685e2cf1c1da72796d7,die/die<det><def><f><sg><nom>,4,4,2,1,0,1,0.5,0.000000,2013-02-28 18:28:01
2,1.0,1362076081,27649635,u:FO,de,en,35a54c25a2cda8127343f6a82e6f6b7d,mann/mann<n><m><sg><nom>,5,4,1,1,1,0,0.0,0.200000,2013-02-28 18:28:01
3,0.5,1362076081,27649635,u:FO,de,en,0cf63ffe3dda158bc3dbd55682b355ae,frau/frau<n><f><sg><nom>,6,5,2,1,1,1,0.5,0.166667,2013-02-28 18:28:01
4,1.0,1362076081,27649635,u:FO,de,en,84920990d78044db53c1b012f5bf9ab5,das/das<det><def><nt><sg><nom>,4,4,1,1,0,0,0.0,0.000000,2013-02-28 18:28:01


In [ ]:
#dataframe.info()

In [ ]:
#dataframe['p_recall'] = dataframe['p_recall'].astype(int)
dataframe['user_id'] = dataframe['user_id'].astype(str)
dataframe['lexeme_id'] = dataframe['lexeme_id'].astype(str)

In [ ]:
sample_data = dataframe[:1000]

In [ ]:
item_response = pd.pivot_table(sample_data, values= "p_recall", index="user_id", columns="lexeme_id")


In [ ]:
import numpy as np

class IRTModel:
    def __init__(self, n_items, n_factors=1, alpha=None, beta=None):
        self.n_items = n_items
        self.n_factors = n_factors
        self.alpha = alpha if alpha is not None else np.random.normal(size=n_items)
        self.beta = beta if beta is not None else np.random.normal(size=(n_factors, n_items))

    def fit(self, X, n_iter=100, learning_rate=0.1):
        old_loglik = float('-inf')
        for i in range(n_iter):
            loglik = self._log_likelihood(X)
            if loglik is not None and loglik > old_loglik:
                old_loglik = loglik
            else:
                break
            gradients = self._compute_gradients(X)
            self.alpha += learning_rate * gradients['alpha']
            self.beta += learning_rate * gradients['beta']

    def _log_likelihood(self, X):
        pass

    def _compute_gradients(self, X):
        pass


In [ ]:
#initiating the model 
model = IRTModel(n_items=len(item_response.columns))

In [ ]:
#fitting the model 
model.fit(item_response.values)

# Print the estimated item difficulty parameters
print("Estimated item difficulties:", model.alpha)

Estimated item difficulties: [ 2.01339084e+00 -5.32966028e-01  6.58201137e-01  4.76803132e-01
  7.20007649e-01  2.50363922e+00 -3.99375536e-01 -1.32523746e-01
 -1.17760808e-01 -2.66938703e-01 -1.47124715e+00  1.36840972e+00
 -2.58447687e-01  3.86588261e-01  1.15464411e+00  9.32401537e-01
  3.38446681e-01 -6.65753194e-01 -1.54083078e+00 -1.05255334e+00
  2.31978595e-01  1.01717892e+00 -3.61873768e-01 -1.05558402e-02
 -1.12874892e+00  6.37633755e-01  7.18696680e-01  7.61143761e-01
 -2.30882659e-01  9.17956991e-01 -2.72763928e-02 -1.04833981e+00
  2.55453066e-02 -9.39445414e-01  4.04528153e-01 -5.07397748e-01
 -2.62760595e-02 -1.72701297e+00 -1.44900114e+00 -5.67031154e-01
  7.09758326e-01 -2.66172447e-01 -4.44556946e-01 -4.72318333e-01
 -4.19011823e-01  7.97200461e-02  2.61390266e-01  8.52492115e-01
  1.19165394e+00  1.43378564e-01  5.57790912e-01 -8.81802877e-02
  1.74931337e+00  2.44625258e+00 -3.38930676e-02  1.30637672e+00
  1.97942513e-01 -2.50176279e-01  3.16992284e-01 -4.47007291e

In [ ]:
import pyirt

In [ ]:
from pyirt import irt

In [ ]:
import numpy as np
from scipy.optimize import minimize

class IRTModel:
    def __init__(self, num_items, num_subjects):
        self.num_items = num_items
        self.num_subjects = num_subjects
        self.item_params = None

    def fit(self, X, y):
        # X: a matrix of shape (num_subjects, num_items)
        # y: a matrix of shape (num_subjects, num_items) representing the response data (0 or 1)

        def neg_log_likelihood(params):
            discrimination = params[:self.num_items]
            difficulty = params[self.num_items:]

            p = 1 / (1 + np.exp(-discrimination * (X - difficulty)))
            return -np.sum(y * np.log(p) + (1 - y) * np.log(1 - p))

        init_params = np.zeros(self.num_items + self.num_subjects)
        results = minimize(neg_log_likelihood, init_params, method='L-BFGS-B')
        self.item_params = results.x

    def predict(self, X):
        discrimination = self.item_params[:self.num_items]
        difficulty = self.item_params[self.num_items:]
        p = 1 / (1 + np.exp(-discrimination * (X - difficulty)))
        return p


In [ ]:
data = dataframe
data['p_recall'] = data['p_recall'].astype(int)

In [ ]:
# Load data
data = data.loc[:, ["user_id", "lexeme_id", "p_recall"]]
# Convert data to 0/1 format
data["p_recall"] = np.where(data["p_recall"] == 1, 1, 0)
data

,user_id,lexeme_id,p_recall
0,u:FO,76390c1350a8dac31186187e2fe1e178,1
1,u:FO,7dfd7086f3671685e2cf1c1da72796d7,0
2,u:FO,35a54c25a2cda8127343f6a82e6f6b7d,1
3,u:FO,0cf63ffe3dda158bc3dbd55682b355ae,0
4,u:FO,84920990d78044db53c1b012f5bf9ab5,1
...,...,...,...
12854221,u:i5D8,d5efc552aaea3109eb5388aa1ec8673d,0
12854222,u:i5D8,a826c47947d68549fa81e19cafa57ba0,0
12854223,u:i5D8,5e29d77697d23070a1fb92eb6c90e9b6,1
12854224,u:i5D8,cdfecc9247566d40bb964a218c54c783,0


In [ ]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

# Load data
data = data.loc[:, ["user_id", "lexeme_id", "p_recall"]]

# Convert data to 0/1 format
data["p_recall"] = np.where(data["p_recall"] == 1, 1, 0)

# Define log-likelihood function for 2PL model
def log_likelihood_2pl(theta, a, b, k, r):
    """
    Compute the log-likelihood function for the 2PL model.

    Parameters:
    -----------
    theta : array-like
        Vector of ability parameters.
    a : array-like
        Vector of discrimination parameters.
    b : array-like
        Vector of difficulty parameters.
    k : array-like
        Vector of guessing parameters.
    r : array-like
        Vector of slipping parameters.

    Returns:
    --------
    log_likelihood : float
        Log-likelihood of the data given the model parameters.
    """
    p = 1 / (1 + np.exp(-a*(theta-b)))
    log_likelihood = np.sum(r*np.log(p) + (1-r)*np.log(1-p) + k*(1-p)*np.log(p) + (1-k)*p*np.log(1-p))
    return log_likelihood

# Define function to optimize log-likelihood
def optimize_log_likelihood(theta, a, b, k, r, args):
    """
    Function to optimize the log-likelihood function.

    Parameters:
    -----------
    theta : array-like
        Vector of ability parameters.
    a : array-like
        Vector of discrimination parameters.
    b : array-like
        Vector of difficulty parameters.
    k : array-like
        Vector of guessing parameters.
    r : array-like
        Vector of slipping parameters.
    args : tuple
        Tuple containing the data.

    Returns:
    --------
    neg_log_likelihood : float
        Negative log-likelihood of the data given the model parameters.
    """
    data = args[0]
    theta = np.array(theta)
    a = np.array(a)
    b = np.array(b)
    k = np.array(k)
    r = np.array(r)
    neg_log_likelihood = -log_likelihood_2pl(theta, a, b, k, r)
    return neg_log_likelihood

# Initialize model parameters
n_items = data["lexeme_id"].nunique()
n_users = data["user_id"].nunique()
theta = np.zeros(n_users)
a = np.ones(n_items)
b = np.zeros(n_items)
k = np.zeros(n_items)
r = np.zeros(n_items)

# Optimize log-likelihood
bounds = [(None, None)]*n_users + [(0, None)]*n_items + [(-5, 5)]*n_items + [(0, 1)]*n_items + [(0, 1)]*n_items
result = minimize(optimize_log_likelihood, x0=list(theta) + list(a) + list(b) + list(k) + list(r),
                  args=(a, b, k, r, (data,)), method='L-BFGS-B', bounds=bounds)
optimized_params = result.x

# Extract ability parameters
theta_hat = optimized_params[:n_users]

# Extract discrimination, difficulty, guessing, and slipping parameters
a_hat = optimized_params[n_users:n_users+n_items]
b_hat


ValueError: ignored

In [ ]:
import stan
from stan import model, fit 

In [ ]:
pip install pystan 3.1.1


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement pystan-3.1.1 (from versions: none)
ERROR: No matching distribution found for pystan-3.1.1


In [ ]:
python3 -m pip install pystan

SyntaxError: ignored

In [ ]:
help(stan)

Help on package stan:

NAME
    stan

PACKAGE CONTENTS
    common
    fit
    model
    plugins

VERSION
    3.3.0

FILE
    /usr/local/lib/python3.8/dist-packages/stan/__init__.py




In [ ]:
stan_code = """
data {
  int<lower=1> J;                     // number of students
  int<lower=1> K;                     // number of questions
  int<lower=1> N;                     // number of observations
  array[N] int<lower=1, upper=J> jj;  // student for observation n
  array[N] int<lower=1, upper=K> kk;  // question for observation n
  array[N] int<lower=0, upper=1> y;   // correctness for observation n
}

parameters {
  real delta;            // mean student ability
  array[J] real alpha;   // ability of student j - mean ability
  array[K] real beta;    // difficulty of question k
}

model {
  alpha ~ std_normal();         // informative true prior
  beta ~ std_normal();          // informative true prior
  delta ~ normal(0.75, 1);      // informative true prior
  for (n in 1:N) {
    y[n] ~ bernoulli_logit(alpha[jj[n]] - beta[kk[n]] + delta);
  }
}

"""

In [ ]:
async def main():
    print(1)
    
await main()

1


In [ ]:
stan_model = stan.build(stan_code)

RuntimeError: ignored

In [ ]:
#2 PL

stan_code = """
data {
  int<lower=1> J;                     // number of students
  int<lower=1> K;                     // number of questions
  int<lower=1> N;                     // number of observations
  array[N] int<lower=1, upper=J> jj;  // student for observation n
  array[N] int<lower=1, upper=K> kk;  // question for observation n
  array[N] int<lower=0, upper=1> y;   // correctness for observation n
}
parameters {
  real mu_beta;                // mean question difficulty
  vector[J] alpha;             // ability for j - mean
  vector[K] beta;              // difficulty for k
  vector<lower=0>[K] gamma;    // discrimination of k
  real<lower=0> sigma_beta;    // scale of difficulties
  real<lower=0> sigma_gamma;   // scale of log discrimination
}
model {
  alpha ~ std_normal();
  beta ~ normal(0, sigma_beta);
  gamma ~ lognormal(0, sigma_gamma);
  mu_beta ~ cauchy(0, 5);
  sigma_beta ~ cauchy(0, 5);
  sigma_gamma ~ cauchy(0, 5);
  y ~ bernoulli_logit(gamma[kk] .* (alpha[jj] - (beta[kk] + mu_beta)));
}

"""
stan_model = stan.build(stan_code)

RuntimeError: ignored

In [ ]:
data = dataframe
J = len(data['user_id'])
K = len(data['lexeme_id'])
N = len(data['mistakes'])

In [ ]:
J

12854226

In [ ]:
import concurrent.futures

def run_model():
    stan_code = """
    data {
      int<lower=1> user_id ;                     // number of Users 
      int<lower=1> K;                     // number of Lexemes 
      int<lower=1> N;                     // number of observations
      array[N] int<lower=1, upper=J> jj;  // student for observation n
      array[N] int<lower=1, upper=K> kk;  // question for observation n
      array[N] int<lower=0, upper=1> y;   // correctness for observation n
    }
    parameters {
      real mu_beta;                // mean question difficulty
      vector[user_id] alpha;             // ability for j - mean
      vector[K] beta;              // difficulty for k
      vector<lower=0>[K] gamma;    // discrimination of k
      real<lower=0> sigma_beta;    // scale of difficulties
      real<lower=0> sigma_gamma;   // scale of log discrimination
    }
    model {
      alpha ~ std_normal();
      beta ~ normal(0, sigma_beta);
      gamma ~ lognormal(0, sigma_gamma);
      mu_beta ~ cauchy(0, 5);
      sigma_beta ~ cauchy(0, 5);
      sigma_gamma ~ cauchy(0, 5);
      y ~ bernoulli_logit(gamma[kk] .* (alpha[jj] - (beta[kk] + mu_beta)));
    }

    """
    stan_model = stan.build(stan_code)

with concurrent.futures.ThreadPoolExecutor() as executor:
    future = executor.submit(run_model)
    result = future.result()


Building...

CRITICAL:httpstan:Exception while compiling `program_code`: `ValueError("Semantic error in '/tmp/httpstan_sxv85m9s/model_z4d43rwo.stan', line 6, column 34 to column 35:\n   -------------------------------------------------\n     4:        int<lower=1> K;                     // number of Lexemes \n     5:        int<lower=1> N;                     // number of observations\n     6:        array[N] int<lower=1, upper=J> jj;  // student for observation n\n                                           ^\n     7:        array[N] int<lower=1, upper=K> kk;  // question for observation n\n     8:        array[N] int<lower=0, upper=1> y;   // correctness for observation n\n   -------------------------------------------------\n\nIdentifier 'J' not in scope.")`
Exception while compiling `program_code`: `ValueError("Semantic error in '/tmp/httpstan_sxv85m9s/model_z4d43rwo.stan', line 6, column 34 to column 35:\n   -------------------------------------------------\n     4:        int<lower=1> K;      

Building: Semantic error:   -------------------------------------------------
     4:        int<lower=1> K;                     // number of Lexemes 
     5:        int<lower=1> N;                     // number of observations
     6:        array[N] int<lower=1, upper=J> jj;  // student for observation n
                                           ^
     7:        array[N] int<lower=1, upper=K> kk;  // question for observation n
     8:        array[N] int<lower=0, upper=1> y;   // correctness for observation n
   -------------------------------------------------

Identifier 'J' not in scope.

ValueError: ignored

In [ ]:
import stan

# Define the Stan model
stan_code = """
data {
  int<lower=1> J;                     // number of students
  int<lower=1> K;                     // number of questions
  int<lower=1> N;                     // number of observations
  array[N] int<lower=1, upper=J> jj;  // student for observation n
  array[N] int<lower=1, upper=K> kk;  // question for observation n
  array[N] int<lower=0, upper=1> y;   // correctness for observation n
}
parameters {
  real mu_beta;                // mean question difficulty
  vector[J] alpha;             // ability for j - mean
  vector[K] beta;              // difficulty for k
  vector<lower=0>[K] gamma;    // discrimination of k
  real<lower=0> sigma_beta;    // scale of difficulties
  real<lower=0> sigma_gamma;   // scale of log discrimination
}
model {
  alpha ~ std_normal();
  beta ~ normal(0, sigma_beta);
  gamma ~ lognormal(0, sigma_gamma);
  mu_beta ~ cauchy(0, 5);
  sigma_beta ~ cauchy(0, 5);
  sigma_gamma ~ cauchy(0, 5);
  y ~ bernoulli_logit(gamma[kk] .* (alpha[jj] - (beta[kk] + mu_beta)));
}
"""

# Define the data
data = {
    'J': 800,
    'K': 20,
    'N': len(data),
    'jj': data.user_id.values.astype(int),
    'kk': data.question.values.astype(int),
    'y': data.p_recall.values.astype(int)
}

# Compile the model
model = stan.build(stan_code, data=data)

# Fit the model
fit = model.sample(num_chains=4)

# Print the summary of the fit
print(fit.summary())


ValueError: ignored